In [ ]:
#Соревнования kaggle https://www.kaggle.com/c/titanic

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab.data_table import DataTable
pd.set_option('display.max_columns', None)
DataTable.max_columns = 30

In [ ]:
#скачиваем данные из github
!wget -O test.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
!wget -O train.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
test_data = pd.read_csv('../content/test.csv')
train_data = pd.read_csv('../content/train.csv')


--2024-08-13 12:05:10--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28210 (28K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  27.55K  --.-KB/s    in 0.002s  

2024-08-13 12:05:10 (11.2 MB/s) - ‘test.csv’ saved [28210/28210]

--2024-08-13 12:05:10--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 

In [ ]:
# 2. Предварительная обработка данных

# Заполнение пропущенных значений
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# Преобразование категориальных данных в числовые
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked'], drop_first=True)

# Выбор признаков для модели
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']
X = train_data[features]
y = train_data['Survived']

# Масштабирование данных
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(test_data[features])
# Разделение данных на обучающую и валидационную выборки
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 3. Создание и компиляция модели
model_0 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_0.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model_0.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5690 - loss: 0.6928 - val_accuracy: 0.7486 - val_loss: 0.5743
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7767 - loss: 0.5460 - val_accuracy: 0.7933 - val_loss: 0.4538
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8074 - loss: 0.4652 - val_accuracy: 0.7989 - val_loss: 0.4175
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8333 - loss: 0.3959 - val_accuracy: 0.8101 - val_loss: 0.4182
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7765 - loss: 0.4708 - val_accuracy: 0.8045 - val_loss: 0.4141
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8037 - loss: 0.4516 - val_accuracy: 0.8101 - val_loss: 0.4203
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8223 - loss: 0.4155 - val_accuracy: 0.8045 - val_loss: 0.4314
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8393 - loss: 0.3932 - val_accuracy: 0.8101 - 

In [ ]:
val_loss, val_accuracy = model_0.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy:.4f}')


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8234 - loss: 0.3981 
Validation accuracy: 0.8045


In [111]:
# 6. Предсказания для тестового набора данных
predictions = model_0.predict(X_test)
predictions = (predictions > 0.5).astype(int)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [112]:

# 7. Подготовка и сохранение файла для отправки на Kaggle

submission = pd.read_csv('/content/drive/MyDrive/датасеты/gender_submission (1).csv')
submission['Survived'] = predictions
submission.to_csv('submission.csv', index=False)

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
